In [1]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

In [2]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('calendar', 'v3', credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                            singleEvents=True, maxResults=100,
                                            orderBy='startTime').execute()
    events = events_result.get('items')

    if not events:
        print('No upcoming events found.')

except HttpError as error:
    print('An error occurred: %s' % error)


Getting the upcoming 10 events


In [3]:
# Prints the start and name of the next 10 events   
color = {
    None:   "🐬",#🈳
    "1":    "🌊",#🟦
    "2":    "🗽",#🈯
    "3":    "🍇",#🟪
    "4":    "🦜",#🆑
    "5":    "🐆",#🟨
    "6":    "🦀",#🟧
    "7":    "🌍",#
    "8":    "🐘",#⬜
    "9":    "🏞️",#
    "10":   "🐢",#🟩
    "11":   "🍎",#🟥
}

for k,e in enumerate(events):
    if 'date' in e['start'].keys():
        events[k]['start']['date'] = datetime.datetime.strptime(events[k]['start']['date'],"%Y-%m-%d")
        events[k]['end']['date'] = datetime.datetime.strptime(events[k]['end']['date'],"%Y-%m-%d")
        
        events[k]['start']['date'] = events[k]['start']["date"].strftime("%d/%m/%Y")
        events[k]['start']['time'] = "All Day"
        events[k]['end']['date'] = events[k]['end']["date"].strftime("%d/%m/%Y")
        events[k]['end']['time'] = "All Day"
        
    elif 'dateTime' in e['start'].keys():     
        events[k]['start']['dateTime'] = datetime.datetime.strptime(events[k]['start']['dateTime'],"%Y-%m-%dT%H:%M:%S+%U:%W")
        events[k]['end']['dateTime'] = datetime.datetime.strptime(events[k]['end']['dateTime'],"%Y-%m-%dT%H:%M:%S+%U:%W")
        
        events[k]['start']['date'] = events[k]['start']["dateTime"].strftime("%d/%m/%Y")
        events[k]['start']['time'] = events[k]['start']["dateTime"].strftime("%H:%M")
        events[k]['end']['date'] = events[k]['end']["dateTime"].strftime("%d/%m/%Y")
        events[k]['end']['time'] = events[k]['end']["dateTime"].strftime("%H:%M")
        
for event in events: 
    print(event['start']['date'], event['start']['time'], "->", event['end']['date'], event['end']['time'], event['summary'])

17/06/2022 13:30 -> 17/06/2022 15:30 EXAM: Sensors from h13.30
18/06/2022 All Day -> 19/06/2022 All Day Ghent
21/06/2022 11:50 -> 21/06/2022 13:25 Flight to Milan (FR 4526)
22/06/2022 All Day -> 23/06/2022 All Day Meeting NearLab - Ke
30/06/2022 All Day -> 01/07/2022 All Day Neuroengineering Exam 1st call
30/06/2022 10:00 -> 30/06/2022 12:00 Esame MAM Ventu
01/07/2022 All Day -> 02/07/2022 All Day Computational Biomechanics Exam 1st call
05/07/2022 All Day -> 06/07/2022 All Day Eliminazione Instagram
18/07/2022 All Day -> 19/07/2022 All Day Computational Biomechanics Exam 2nd call
20/07/2022 All Day -> 21/07/2022 All Day Laurea Ventu
21/07/2022 All Day -> 22/07/2022 All Day Neuroengineering Exam 2nd call
26/07/2022 All Day -> 27/07/2022 All Day Biosensors Exam 2nd call


In [4]:
from telegram.ext import (Updater, 
    CommandHandler,
    MessageHandler,
    CallbackQueryHandler,
    ConversationHandler,
    CallbackContext,    
    Filters
)
import datetime
with open("telegramapi.txt") as f:
        tokens = f.readlines()
bot_token = tokens[0].strip("\n")
updater = Updater(token=bot_token, use_context=True)
        

        
print(">> "+datetime.datetime.now().strftime("[%d/%m/%Y-%H:%M:%S]-BOT: ")+
        " Loading Telegram bot: ",end="")
dispatcher = updater.dispatcher
print("SUCCESS") if updater!=0 else print("FAIL")

# dispatcher.add_handler(CommandHandler('send_message', send_message))
# dispatcher.add_handler(CommandHandler('start', printid))

# jobqueue = updater.job_queue
# jobqueue.run_repeating(
#     callback=send_message,
#     interval=1,
# )

def center(text, w,sep='_'):
    ext = sep
    left = (w-len(text))//2
    right = w-left-len(text)
    return left*ext+ text + right*ext

MENU = 0
NEW_EVENT = 1
NEW_EVENT_FINAL = 2
LINEWIDTH = 36
BT = "`"
SEP = "_"
import telegram
from telegram import ReplyKeyboardMarkup
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext import CallbackQueryHandler

def modifychosen_callback(update, context):
    return


def modifyevent_callback(update, context):
    events_keyboard = []
    for e in events:
        events_keyboard.append(
            [InlineKeyboardButton(color[e['colorId']]+" "+e['summary'], callback_data=modifychosen_callback)]
        )
        
        update.message.reply_text(
            "Which event do you want to modify?",
            parse_mode=telegram.ParseMode.MARKDOWN,
        )
    return MENU

def addevent_callback(update,context):
    query = update.callback_query
    context.bot.send_message(
        chat_id=update.effective_chat.id, 
        text="Type the details of the event you want to create:\n"+
        "_Start Date_\n"+
        "_Start Time_\n"+
        "_Name_\n"+
        "_Description_\n"+
        "(write '.' in the field you want to ignore)",
        parse_mode=telegram.ParseMode.MARKDOWN,
    )
    return NEW_EVENT

def newevent_callback(update, context):
    query = update.callback_query
    eventtext = update.message.text.split("\n")
    global event
    event = events[-1]
    event['date']['start'] = eventtext[0] if eventtext[0]!="." else None
    event['time']['start'] = eventtext[1] if eventtext[1]!="." else None
    event['summary'] = eventtext[2] if eventtext[2]!="." else None
    event['description'] = eventtext[3] if eventtext[3]!="." else None
    
    events.append((event))
    symbols = [
        [InlineKeyboardButton(color[None],callback_data=color[None]),
         InlineKeyboardButton(color['1'],callback_data=color['1']),
         InlineKeyboardButton(color['2'],callback_data=color['2']),
         InlineKeyboardButton(color['3'],callback_data=color['3']),],
        
        [InlineKeyboardButton(color['4'],callback_data=color['4']),
         InlineKeyboardButton(color['5'],callback_data=color['5']),
         InlineKeyboardButton(color['6'],callback_data=color['6']),
         InlineKeyboardButton(color['7'],callback_data=color['7']),],
        
        [InlineKeyboardButton(color['8'],callback_data=color['8']),
         InlineKeyboardButton(color['9'],callback_data=color['9']),
         InlineKeyboardButton(color['10'],callback_data=color['10']),
         InlineKeyboardButton(color['11'],callback_data=color['11']),],
    ]
    
    reply_markup = InlineKeyboardMarkup(symbols)
    context.bot.send_message(
        chat_id=update.effective_chat.id, 
        text="Choose the category",
        parse_mode=telegram.ParseMode.MARKDOWN,
        reply_markup = reply_markup
    )
    return finalizeevent_callback(update,context)
    # event = service.events().insert(calendarId='primary', body=event).execute()
    
def finalizeevent_callback(update, context):
    event['colorID'] = update.callback_query.data
    events.append((event))
    return MENU
    
def showevents_callback(update, context):
    for e in events:
        # One day events
        if e['start']['date'] == e['end']['date']:
            
            # All day events
            if e['start']['time'] == "All Day":
                msg_text = "*"+e['start']['date']+"  All Day*\n"
                msg_text += color[e.get("colorId")]+" "+e['summary']+"\n\n"
                if e.get("description") is not None: msg_text += e.get('description')[:100]
                
            else:
                msg_text = "*"+e['start']['date']+"*-_"+e['start']['time']+"_ ➡️ *"+ e['end']['date']+"*-_"+e['end']['time']+"_\n"
                msg_text += color[e.get("colorId")]+" "+e['summary']+"\n\n"
                if e.get("description") is not None: msg_text += e.get('description')[:100]
                
        # Multiple day events
        else:
            msg_text = "*"+e['start']['date']+" ➡️ "+ e['end']['date']+"*\n"
            msg_text += color[e.get("colorId")]+" "+e['summary']+"\n\n"
            if e.get("description") is not None: msg_text += e.get('description')[:100]
            
        update.message.reply_text(
            msg_text,
            parse_mode=telegram.ParseMode.MARKDOWN,
        )
    buttons = [
        [InlineKeyboardButton("🆕Add Event",callback_data="addevent_callback"),
        InlineKeyboardButton("♻️Modify Event",callback_data="modifyevent_callback"),
        InlineKeyboardButton("💣Delete Event",callback_data="deleteevent_callback")],
    ]
    reply_markup = InlineKeyboardMarkup(buttons)
    update.message.reply_text(
        "Next",
        parse_mode=telegram.ParseMode.MARKDOWN,
        reply_markup = reply_markup
    )
    # return MENU


def tasks_callback(update,context):
    client=context.dispatcher.user_data['client']
    user_id = update.message.from_user['id']
    menu_options = [
        ['Events','Tasks']
    ]

    update.message.reply_text(
        text="Showing Tasks NOW",
        # chat_id=update.effective_chat.id,
        parse_mode=telegram.ParseMode.HTML,
        reply_markup=ReplyKeyboardMarkup(
            menu_options, 
            one_time_keyboard=True,
            resize_keyboard=True,
            )
    )
    return MENU

def menu(update, context):
    user_id = update.message.from_user['id']
    print(">> "+datetime.datetime.now().strftime(f"[%d/%m/%Y-%H:%M:%S]-{user_id}: ")+
        " MENU request ")
    
    menu_options = [
        ['Events','Tasks']
    ]

    update.message.reply_text(
        "Select from the menu:  ",
        reply_markup=ReplyKeyboardMarkup(
            menu_options, 
            one_time_keyboard=True,
            resize_keyboard=True,
        ),
    )
    return MENU

conv_handler = ConversationHandler(
        entry_points=[CommandHandler('menu',menu)],
        states={
            MENU: 
                [
                MessageHandler(Filters.regex('^(Events)$'), showevents_callback),
                MessageHandler(Filters.regex('^(Tasks)$'), tasks_callback),
                MessageHandler(Filters.text & ~Filters.command, newevent_callback),
                ],
            NEW_EVENT:
                [
                MessageHandler(Filters.text & ~Filters.command, addevent_callback),
                ],
            NEW_EVENT_FINAL:
                [
                MessageHandler(Filters.text & ~Filters.command, finalizeevent_callback),
                ]
        },
        fallbacks=[CommandHandler('menu', menu)]
)
dispatcher.add_handler(conv_handler)
dispatcher.add_handler(CommandHandler('menu', menu))
dispatcher.add_handler(CallbackQueryHandler(addevent_callback))

updater.start_polling()
updater.idle()

>> [16/06/2022-23:24:18]-BOT:  Loading Telegram bot: SUCCESS


: 

In [ ]:
buttons = [
    [InlineKeyboardButton("Add Event",callback_data=addevent_callback)],
    # [InlineKeyboardButton("Modify Event",callback_data=modifyevent_callback)],
    # [InlineKeyboardButton("Delete Event",callback_data=deleteevent_callback)],
]
InlineKeyboardMarkup(buttons)

In [ ]:
events_keyboard = []
for e in events:
    print(e['summary'])
    if 'date' in e['start'].keys():
        events_keyboard.append([
            InlineKeyboardButton(e['summary'], 
                                    callback_data= "f")]
        )
    elif 'dateTime' in e['start'].keys():
        events_keyboard.append([
            InlineKeyboardButton(e['summary'], 
                                    callback_data= "f")]
        )
events_keyboard[0]

EXAM: Sensors from h13.30
Ghent
Flight to Milan (FR 4526)
Meeting NearLab - Ke
Neuroengineering Exam 1st call
Esame MAM Ventu
Computational Biomechanics Exam 1st call
Eliminazione Instagram
Computational Biomechanics Exam 2nd call
Laurea Ventu
Neuroengineering Exam 2nd call
Biosensors Exam 2nd call
